In [1]:
import itolapi
from itolapi import Itol
import os
from pathlib import Path
import sys

import pandas as pd
import toytree       # a tree plotting library
import toyplot       # a general plotting library
import numpy as np   # numerical library
import os
from Bio import SeqIO
from Bio import Entrez
import subprocess
import toyplot.png
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import subprocess

In [2]:
import sys
sys.path.insert(0, '/n/home11/rkapoor')
import tax_pkg
from tax_pkg import taxid
from tax_pkg import accession2taxid

In [3]:
import multiprocessing as mp

In [ ]:

def get_species(x):
    try:
        
        l=taxid.get_lineage(x,{})
        ls=l['species']
        if type(ls) is str:
            s=ls
        else:
            s=list(l['species'])[0]
    except:
        s=x

    return s
colors=["blue","brown","green","teal","pink","orange","black","red","yellow","magenta"]
cmap={c:mcolors.to_hex(c) for c in colors}


In [ ]:
def prep_for_upload(n):
    
    try:
        subprocess.run(["python3", "MAD/MAD_root.py", f"hmmer_phylo_data/{n}"] )
        tre=toytree.tree(f'hmmer_phylo_data/{n}/rev_aa.treefile.rooted2')
    except:
        tre=toytree.tree(f'hmmer_phylo_data/{n}/rev_aa.treefile')
    labs=tre.get_tip_labels()
    labs2=[]
    for x in labs:
        if len(x.split("_"))==4:
            labs2.append(x.split("_")[0]+"_"+x.split("_")[1])
        else:
            labs2.append(x.split("_")[0])

    df12=pd.read_csv(f"hmmer_phylo_data/{n}/phylo_tax.tsv",sep="\t")
    df12=df12.set_index("target_name")
    df12=df12.loc[labs2,:]
    df12.color=df12.color.replace("None","black")
    df12.color=df12.color.astype(str).replace("nan","black")
    for index, row in df12[df12.chimera==True].iterrows():
        df12.loc[index,"color"]="red"
    df12.loc[n.split(";")[1],"color"]="magenta"
    
    for index, row in df12.iterrows():
        df12.loc[index,"color2"]=cmap[row.color]
    df12["label"]=labs
    df12=df12.set_index("label")
        
    f=open(f"hmmer_phylo_data/{n}/colorstrip.txt","w")
    f1=open("color_strip_dataset.txt","r")
    for x in f1.readlines():
        f.write(x)
    f1.close()
    for index, row in df12.iterrows():
        f.write(index+" "+row.color2)
        f.write("\n")
    f.close()
    f=open(f"hmmer_phylo_data/{n}/labels.txt","w")
    f1=open("label_template.txt","r")
    for x in f1.readlines():
        f.write(x)
    f1.close()
    for index, row in df12.iterrows():
        try:
            f.write(index+","+str(row.sci_name).replace(",","_")+" ("+row.phylum.replace(",","_")+"_"+row.order.replace(",","_")+")"+",-1,#000000,normal,1,0")
            f.write("\n")
        except:
            no=1
    f.close()
    
    
    f=open(f"hmmer_phylo_data/{n}/labels2.txt","w")
    f1=open("label_template2.txt","r")
    for x in f1.readlines():
        f.write(x)
    f1.close()
    for index, row in df12.iterrows():
        try:
            f.write(index+","+str(row["description of target"]).replace(",","_")+",-1,#000000,normal,1,0")
        except:
            no=1
        f.write("\n")
    f.close()
    


In [ ]:
def upload(n,project,workspace):
    
    test = Itol()
    try:
        current_dir = Path(os.path.realpath(f"hmmer_phylo_data/{n}/rev_aa.treefile.rooted2")).parent
        tree = current_dir/"rev_aa.treefile.rooted2"
        test.add_file(tree)
    except:
        current_dir = Path(os.path.realpath(f"hmmer_phylo_data/{n}/rev_aa.treefile")).parent
        tree = current_dir/"rev_aa.treefile"
        test.add_file(tree)
        upload_root(n,project,workspace)
        
    root_path = current_dir.parent
    sys.path.append(str(root_path))
    

    # Set the tree file


    
    test.add_file(current_dir / 'colorstrip.txt')
    test.add_file(current_dir / 'labels.txt')
    test.add_file(current_dir / 'labels2.txt')
    # Add parameters

    test.params['APIkey'] = 'nsmHyk7faBGaJty82r9m5Q'
    test.params['treeName'] = n
    test.params['projectName'] = project
    test.params['workspaceName'] = workspace
  
    # Submit the tree
    good_upload = test.upload()
    if not good_upload:
        print('There was an error:' + test.comm.upload_output)
        sys.exit(1)
    else:
        f=open("good_uploads_7_13.txt","a")
        f.write(n)
        f.write("/n")
        f.close()


In [ ]:
def upload_root(n,project,workspace):
    current_dir = Path(os.path.realpath(f"hmmer_phylo_data/{n}/rev_aa.treefile.rooted")).parent
    tree = current_dir/"rev_aa.treefile.rooted"

        
    root_path = current_dir.parent
    sys.path.append(str(root_path))
    test = Itol()

    # Set the tree file


    test.add_file(tree)
    test.add_file(current_dir / 'colorstrip.txt')
    test.add_file(current_dir / 'labels.txt')
    test.add_file(current_dir / 'labels2.txt')
    # Add parameters

    test.params['APIkey'] = 'nsmHyk7faBGaJty82r9m5Q'
    test.params['treeName'] = n
    test.params['projectName'] = "not_rooted_"+project
    test.params['workspaceName'] = workspace
    
    # Submit the tree
    good_upload = test.upload()
    if not good_upload:
        print('There was an error:' + test.comm.upload_output)
        sys.exit(1)
    else:
        f=open("good_uploads_7_13.txt","a")
        f.write(n+"~root")
        f.write("/n")
        f.close()

    

In [ ]:
def upload_Wrapper(ps):
    n,project,workspace=ps
    try:
        prep_for_upload(n)
        upload(n,project,workspace)
        
    except:
        f=open("fails7_13","w")
        f.write(n)
        f.write("/n")
        f.close()
        

In [4]:
lrs=[x for x in os.listdir("hmmer_phylo_data") if "HGT" in x and "rev_aa.treefile" in os.listdir("hmmer_phylo_data/"+x) ]

In [ ]:
with mp.Pool(62) as p:
    results = p.map(upload_Wrapper, [[n,"HGT_MAD","Arthropod_HGT_chimeras"] for n in set(lrs)])

In [ ]:
lrs=[x for x in os.listdir("hmmer_phylo_data") if "Meta" in x and "rev_aa.treefile" in os.listdir("hmmer_phylo_data/"+x) ]

In [ ]:
with mp.Pool(62) as p:
    results = p.map(upload_Wrapper, [[n,"HGT_MAD","Arthropod_HGT_chimeras"] for n in set(lrs)])

In [6]:
n=lrs[0]

In [9]:
tre=toytree.tree(f'hmmer_phylo_data/{n}/rev_aa.treefile.rooted2')
df12=pd.read_csv(f"hmmer_phylo_data/{n}/phylo_tax.tsv",sep="\t")

labs=tre.get_tip_labels()
labs2=[]
for x in labs:
    if len(x.split("_"))==4:
        labs2.append(x.split("_")[0]+"_"+x.split("_")[1])
    else:
        labs2.append(x.split("_")[0])
df12=df12.set_index("target_name")
df12=df12.loc[labs2,:]
df12.color=df12.color.replace("None","black")
df12.color=df12.color.astype(str).replace("nan","black")

AttributeError: 'DataFrame' object has no attribute 'color'

In [10]:
df12

,Unnamed: 0,species,target_accession,tlen,query name,accession,qlen,E-value,score1,bias1,...,bias,hmmfrom,hmmto,alifrom,alito,envfrom,envto,acc,description of target,chimera
target_name,,,,,,,,,,,,,,,,,,,,,
XP_049294175.1,83,Anopheles funestus,-,243.0,"GCF_905220565.1;XP_045456871.1;HGT_(1,71)",-,72.0,7.300000e-16,72.3,0.0,...,0.0,4.0,70.0,166.0,232.0,163.0,233.0,0.96,uncharacterized protein LOC125769485 [Anophele...,False
AAL56548.1,84,Anopheles gambiae,-,942.0,"GCF_905220565.1;XP_045456871.1;HGT_(1,71)",-,72.0,1.100000e-14,68.5,0.1,...,0.0,4.0,70.0,865.0,931.0,862.0,932.0,0.96,"pol polyprotein, partial [Anopheles gambiae]",False
XP_049464912.1,82,Anopheles coluzzii,-,246.0,"GCF_905220565.1;XP_045456871.1;HGT_(1,71)",-,72.0,4.400000e-15,69.8,0.0,...,0.0,4.0,71.0,168.0,235.0,165.0,236.0,0.97,uncharacterized protein LOC125907333 [Anophele...,False
XP_011687010.1,1700,Wasmannia auropunctata,-,880.0,"GCF_905220565.1;XP_045456871.1;HGT_(1,71)",-,72.0,2.600000e-16,73.7,1.5,...,1.5,3.0,67.0,804.0,868.0,802.0,871.0,0.94,PREDICTED: uncharacterized protein LOC10544945...,False
KAG5678653.1,1229,Polypedilum vanderplanki,-,536.0,"GCF_905220565.1;XP_045456871.1;HGT_(1,71)",-,72.0,9.300000e-17,75.2,0.9,...,0.9,3.0,69.0,462.0,528.0,460.0,530.0,0.96,hypothetical protein PVAND_008309 [Polypedilum...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XP_003669679.1,958,Naumovozyma dairenensis CBS 421,-,288.0,"GCF_905220565.1;XP_045456871.1;HGT_(1,71)",-,72.0,3.200000e-18,79.8,0.5,...,0.5,1.0,72.0,214.0,285.0,214.0,285.0,0.98,hypothetical protein NDAI_0D01220 [Naumovozyma...,False
XP_022466944.1,790,Kazachstania naganishii CBS 8797,-,1548.0,"GCF_905220565.1;XP_045456871.1;HGT_(1,71)",-,72.0,4.400000e-14,66.6,0.3,...,0.3,1.0,68.0,1471.0,1538.0,1471.0,1541.0,0.95,hypothetical protein KNAG_0L00780 [Kazachstani...,False
XP_013190478.1,73,Amyelois transitella,-,1479.0,"GCF_905220565.1;XP_045456871.1;HGT_(1,71)",-,72.0,1.600000e-14,68.0,0.1,...,0.1,1.0,69.0,1288.0,1356.0,1288.0,1358.0,0.96,PREDICTED: retrovirus-related Pol polyprotein ...,False
